In [1]:
import dev_wtp_io_utils

SyntaxError: invalid syntax (dev_wtp_io_utils.py, line 128)

In [7]:
output_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'

In [8]:
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(output_path)

subject 040, subject 039, subject 168, subject 063, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of length 2

BidDuration: array of length 2

FoodPic: array 

In [4]:
run_event_df_list

,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
0,1,3.000000,6.114502,6.558085,MacNCheese.bmp,1,unhealthy_liked,unhealthy,liked,3,5,3.000000,12.672587,run1,2,DEV040
1,2,3.000000,18.713320,6.532640,Strawberries.bmp,2,healthy_liked,healthy,liked,4,7,1.780283,25.245960,run1,2,DEV040
2,3,1.780283,30.069421,6.519313,RedApple.bmp,3,healthy_liked,healthy,liked,4,6,0.095117,36.588734,run1,2,DEV040
3,4,0.095117,39.713532,6.532611,Fritos.bmp,4,unhealthy_disliked,unhealthy,disliked,1,5,2.680966,46.246143,run1,2,DEV040
4,5,2.680966,51.967374,6.536582,ruffles_cheddar.bmp,5,unhealthy_liked,unhealthy,liked,4,NULL,0.319097,58.503956,run1,2,DEV040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,28,1.000000,206.375708,6.532626,Dots_one.bmp,28,unhealthy_disliked,unhealthy,disliked,1,2,1.000000,212.908334,run2,3,DEV138
28,29,1.000000,213.991678,6.533641,VeggieStraws.bmp,29,healthy_disliked,healthy,disliked,1,2,1.000000,220.525319,run2,3,DEV138
29,30,1.000000,221.592464,6.532627,cranberries.bmp,30,healthy_disliked,healthy,disliked,1,1,1.000000,228.125091,run2,3,DEV138
30,31,1.000000,229.192518,6.532579,Peach.bmp,31,healthy_liked,healthy,liked,3,3,1.000000,235.725096,run2,3,DEV138


### On ben's local machine

In [31]:


def get_comprehensive_run_data_from_mat_dir(mat_dir_path):
    """returns three objects: 
    (1) a run_list (containing a list of runs)
    (2) an event list (containing a list of events)
    (3) an event stage list ( containing a list of event stages divided up by stage)
    """
    run_list = []
    run_event_df_list = []
    run_event_stage_df_list = []

    for subj_mat_dir_pathname in glob.glob(mat_dir_path + 'DEV*'):
        #get the subject folder name
        subj_output_folder_name = os.path.basename(subj_mat_dir_pathname)
        #and get the three digit number that is part of it
        subj_3_digit_number = re.search('DEV(\\d*)',subj_output_folder_name,re.IGNORECASE)[1]

        wave_pathname_list = glob.glob(mat_dir_path + subj_output_folder_name + '/DEV.' + subj_3_digit_number + '.?.mat')

        print('subject ' + subj_3_digit_number,end=', ')
        #iterate through each wave for this subject.
        for wave_pathname in wave_pathname_list:
            #print(wave_pathname)
            subj_regex_code = 'DEV.' + subj_3_digit_number + '.(\\d*).'
            wave_name = re.search(subj_regex_code,os.path.basename(wave_pathname),re.IGNORECASE)[1]

            #we're really just wanting wave 1 right now
            #but let's do all waves because it'll be easier to knock them out now
            #rather than trying to handle them all later.
            #wave1 = glob.glob(mat_dir_path + subj_output_folder_name + '/DEV.' + subj_3_digit_number + '.1.mat')

            #load the wave file for this wave.
            #wave_file = scipy.io.loadmat(wave_pathname,simplify_cells=True)
            wave_file = scipy.io.loadmat(wave_pathname,struct_as_record=True,squeeze_me=True)
            return(wave_file)
            runs = list(filter(re.compile("run*").match,wave_file['Data'].keys()))


            for run in runs:
                run_object = wave_file['Data'][run]

                #before we do this run, let's check that it is complete
                #if it is abridged a known way, print a specific notification
                if set(run_object.keys())==set(['time']):
                    print("\nSUBJ " + subj_3_digit_number + ", " + wave_name +", " + run)
                    print("no data for this run.")
                    continue
                #if it's missing data but not in any way we expect, print some info and continue
                if len(run_object.keys())!=20:
                    print("\n\nSUBJ " + subj_3_digit_number + ", " + wave_name +", " + run)
                    print("Data for this run is not in the expected format or is missing. Skipping this run.")


                    for k,v in run_object.items():
                        print(k,end=': ')
                        if isinstance(v,np.ndarray):
                            print('array of length ' + str(len(v)))
                        else:
                            print(v)
                        print('')

                    continue

                #create run object
                try:
                    #pick out the keys we want for the run object
                    run_obj_simplified = {
                        ro_key: run_object[ro_key] 
                        if ro_key in run_object.keys() 
                        else None 
                        for ro_key in ['StartTime','EndTime','time']
                    }
                    run_obj_simplified['run'] = run
                    run_obj_simplified['wave'] = wave_name
                    run_obj_simplified['subject'] = 'DEV' + subj_3_digit_number
                    run_list=run_list + [run_obj_simplified]
                except:
                    print(run_object)
                    raise

                #create event list
                run_event_df = get_run_event_df(run_object)
                run_event_df['run'] = run
                run_event_df['wave'] = wave_name
                run_event_df['subject'] = 'DEV' + subj_3_digit_number
                run_event_df_list = run_event_df_list + [run_event_df]


                #create event stage list
                event_stage_df = get_run_event_stage_df(run_object)
                event_stage_df['run'] = run
                event_stage_df['wave'] = wave_name
                event_stage_df['subject'] = 'DEV' + subj_3_digit_number
                run_event_stage_df_list = run_event_stage_df_list + [event_stage_df]


                

    run_event_stage_across_runs_df = pd.concat(run_event_stage_df_list)
    run_event_across_runs_df = pd.concat(run_event_df_list)
    run_list_df = pd.DataFrame(run_list)
    
    #tidy up some stuff
    run_event_across_runs_df.wave = run_event_across_runs_df.wave.astype(int)
    #run_event_across_runs_df.likert_rating = run_event_across_runs_df.likert_rating.astype(int)
    run_event_stage_across_runs_df.wave = run_event_stage_across_runs_df.wave.astype(int)
    run_list_df.wave = run_list_df.wave.astype(int)
    
    return(run_list_df,run_event_across_runs_df,run_event_stage_across_runs_df)



In [32]:
import numpy as np
import pandas as pd
import re
import glob
import os
import scipy.io


In [33]:
#output_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
output_path = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/WTP/output/'

In [36]:
x = get_comprehensive_run_data_from_mat_dir(output_path)

subject 089, 

In [46]:
x['Data']

IndexError: too many indices for array

In [35]:
scipy.io.loadmat(

SyntaxError: unexpected EOF while parsing (<ipython-input-35-5c16fbf16592>, line 1)